In [ ]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

# Code testing some notions

* manual testing vs automated testing
* unit testing vs integration testing
* when to test
* TDD or TFD
* a special note on testing notebooks
  

# Python testing with pytest

## 1 Getting started with pytest

pytest is basically a wrapper around the Python native __assert__ keyword.

In [ ]:
def test_passing():
    assert (1, 2, 3) == (1, 2, 3)

test_passing()

In [ ]:
def test_failing():
    assert (1, 2, 3) == (3, 2, 1)

test_failing()

**Installing pytest**

- ```python -m venv whatevername```
- ```whatevername\scripts\activate.bat```
- ```pip install pytest```

**Running pytest**
- ```cd /path/to/code/ch1```
- ```pytest test_one.py```
- ```pytest test_two.py```
- ```pytest -v test_two.py```
- ```pytest``` (pytest will look for all files containing tests in the current working directory)
- ```pytest --tb=no``` (not traceback)
- ```pytest --tb=no test_one.py, test_two.py``` (this actually does the same...)
- ```cd ..```
- ```pytest --tb=no ch1``` (using a path)
- ```pytest -v ch1/test_one.py::test_passing``` (running one specific test function)

**Test discovery**
- Test files should be name ```test_<something>.py``` or ```<something>_test.py```
- Test methods and functions should be named ```test_<something>```
- Test classes should be named ```Test<something>```

**Test Outcomes**
- PASSED (.)
- FAILED (F)
- SKIPPED (s) (```@pytest.mark.skip()``` or ```@pytest.mark.skipif()``` decorators)
- XFAIL (x) (```@pytest.mark.xfail()```)
- XPASS (X)
- ERROR (E)

## 2 Writing test functions

Installing the sample application

- ```pip install ./code/cards_proj/```

Lets play around with the cards todo list application:

- ```cards add do something --owner wessel```
- ```cards add do something else```
- ```cards``` (retrieve a list of todos)
- ```cards update 2 --owner wessel```
- ```cards```
- ```cards start 1```
- ```cards finish 1```
- ```cards start 2```
- ```cards```
- ```cards delete 1```
- ```cards```
  

**Writing knowledge-building tests**

Cards is a three layered application: CLI API and DB. The data structure used to pass infor between the CLI and the API is a class called Card:

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Card:
    summary: str = None
    owner: str = None
    state: str = 'todo'
    id: int = field(default=None, compare=False)

    @classmethod
    def from_dict(cls, d):
        return Card(**d)

    def to_dict(self):
        return asdict(self)


Have a look at ```ch2\test_card.py

then run the tests:

- ```pytest test_card.py```

**Using assert statements**

| pytest | unittest |
| :- | :- |
| assert something | assertTrue(something) |
| assert not something | assertFalse(something) |
| assert a == b | assertEqual(a, b) |
| assert a != b | assertNotEqual(a, b) |
| assert a is None | assertIsNone(a) |
| assert a is not None | assertIsNotNone(a) |
| assert a <= b | assertLessEqual(a, b) |


show ```ch2/test_card_fail.py```

then run:

- ```pytest test_card_fail.py```

ok, with the ```-vv``` flag we get more info:

- ```pytest -vv test_card_fail.py```

To see what Python itself returns when it encounters an assertion failure run

- ```python test_card_fail.py```

**Failing with ```pytest.fail()``` and Exceptions**

A test will fail if:
- an assert statement fails -- AssertionError
- the test code calls ```pytest.fail()```
- any other exception is raised

In [ ]:
import pytest

def test_with_fail():
    pytest.fail()
test_with_fail()

Let's see what that looks like from pytest, but first a look at the code:

```ch2/test_alt_fail.py```

Run:

* ```pytest test_alt_fail.py```

No assertion rewriting here, so when can this be useful then? Well, in the case of Assertion Helper Functions they might. Have a look at ```ch2/test_helper.py``` and see when this is useful. Then run te code:

* ```pytest test_helper.py```

So sometimes something is better than nothing...

**Testing for Expected Exceptions**

Have a look at the code in ```ch2\test_experiment.py```

Then run it:

* ```pytest --tb=short test_experiment.py```

Using ```pytest.raises()``` we can check for the occurance of an exception. Have a look at the ```test_no_path_raises()``` function in ```ch2/test_exceptions.py```.

Then run:

* ```pytest test_exception.py::test_no_path_raises```


## Structuring Test Functions

The best pratices for structuring functions can be called as follows:

1. Arrange - Act - Assert
2. Given - When - Then

Have a look at ```ch2\test_structure.py```

Keep in mind that a test function should only test for one behavior.

**Grouping tests with classes**

Have a look a ```ch2\test_classes.py```. Then run the code:

* ```pytest -v test_classes.py::TestEquality``` (the class is called)

* ```pytest -v test_classes.py::TestEquality::test_equality``` (a specific method is called)

All OOP features are applicable to test classes but be very reluctant to apply them!!!

## 3 pytest Fixtures

**<font color='red'>!!!First present the decorators notebook!!!</font>**

Have a look at ```ch3\test_fixtures.py``` and then run it:

* ```pytest test_fixtures.py::test_some_data```

**Using fixtures for Setup and Teardown**

Let's test the count functionality of the Card program. Have a first try with ```ch3\test_count_initial.py``` and point to the problems.

Then we have a look at ```ch3\test_count.py```, notice the setup and teardown code and the yield in between. Also the teardown code will run whatever the outcome of the test will be.

Run the following:

- ```pytest test_count.py::test_empty```

**Tracing Fixture Execution with --setup-show**

run:

```pytest --setup-show test_count.py```

**Specifying Fixture Scope**

Have a look at ```ch3\test_mod_scope.py```, then run it":

* ```ytest --setup-show test_mod_scope.py```

Scope can be:

- function (default)
- class
- module
- package
- session

**Sharing fixtures through conftest.py**

Have a look at: ```ch3\a\conftest.py``` and ```ch3\a\test_count.py``` then run:

- ```cd a```
- ```pytest --setup-show test_count.py```

*Don't import conftest.py*

**Finding where fixtures are defined**

run: ```pytest --fixtures -v```

Mmmm, that's a lot of fixtures...

To be more specific use: ```pytest --fixtures-per-test test_count.py::test_empty```

### Multiple fixture levels

Have a look at ```ch3\a\test_three.py```

run: ```pytest -v test_three.py```

and now run ```pytest -v --tb=line```, that wasn't going to well

Have a look at ```ch3\b\conftest.py``` for a fix

run: 

- ```cd ..\b```
- ```pytest --setup-show```

## 4 Builtin Fixtures

Some setup and teardown tasks are so common that the fixtures to realize these are a standard part of pytest:

- ```tmp_path``` to create a function scoped temporary directory
- ```tmp_path_factory``` to create session scoped temporary directories
- ```capsys``` to capture output to stdout and stderr
- ```monkeypatch``` to modify objects, dictionaries, environment variables, cwd

## 5 Parametrization

Let's have a look at generating test by parametrizing functions and fixtures.

We want to test the following method from cards\api.py:

In [ ]:
def finish(self, card_id: int):
    '''Set a card state to 'done'.'''
    self.update_card(card_id, card(state="done"))

Have a look at the code in ```ch5\test_finish.py``` and the run it:

- ```pytest -v test_finish.py```

This seems a bit verbose for three very similar tests. Let's look at ```ch5\test_finish_combined``` for a possible solution. An then run it:

- ```pytest -v test_finish_combined```

Mmm, three test now look like one, difficult to understand what is going on, difficult to debug when something goes wrong. And as soon as one assert fails the remaining asserts won't be run...

So let's parametrize things. Have a look at ```ch5\test_func_param.py```, and then run it:

- ```pytest -v test_func_param.py::test_finish```

That went pretty well, however changing the summary for every test is redundant for this test, so we can get rid of it. Have a look at ```ch5\test_func_param.py``` again and no at the ```test_finish_simple()``` function. And then run it:

- ```pytest -v test_func_param.py::test_finish_simple```

We can also parametrize at the fixture level. Have a look at ```ch5\test_fix_param.py```. And then run it:

- ```pytest -v test_fix_param.py```

That looks just about the same with a bit more code. Parametrizing fixtures is useful when you need to setup and teardown stuff for each set of parameters. And you are also able to share parameters between functions.

## 6 Markers

pytest markers are like tags or labels that tell there's something special about a specific test.

```@pytest.mark.slow``` can be used to tag long running tests which need to be omitted when in a hurry.

Some builtin markers modify the behavior of tests like ```@pytest.mark.parametrize``` from the previous chapter.

Some builtin markers:
- ```@pytest.mark.filterwarning```
- ```@pytest.mark.skip```
- ```@pytest.mark.skipif```
- ```@pytest.mark.xfail```
- ```@pytest.mark.parametrize```

Custom markers can be handy to run only a portion of the tests. Have a look at ```ch6\smoke\test_start.py```, and then run it:

- ```pytest -v -m smoke test_start.py```

## 7 Strategy

### Determining Test Scope

Different projects have different test goals and requirements
- Security
- Performance
- Workload (scaling)
- Input validation

Testing Enough to sleep at night

### Evaluating and prioritizing the features to test

- Recent
- Core or USPs
- Risk
- Problematic
- Expertise

## 8 Configuration files

## 9 coverage

Coverage is a measure of how much of your code is being covered by tests. **Line coverage** is calculated by dividing the number of lines of code touched by tests by the total number of lines of code.
**Branch coverage** tells you wether all paths through the code are taken or not.

Keep in mind that coverage doesn't tell you anyting about test quality...

In Python ```Coverage.py``` is the prefered tool to measure code coverage. ```pytest-cov``` is a pytest pluging which makes working with ```coverage.py``` in pytest easier.

## 10 Mocking

Mocking means immitating parts of the system for testing purposes. A unit under test might have dependecies on other (complex) objects. To isolate the behavior of the unit under test you replace the other parts by mocks that simulate the behavior of the real objects. You can also look at it as a crash test dummy.

Within Python the ```mock``` package is used for this purpose. It ships with the standard library as ```unittest.mock```

## 11 Continuous integration

CI is the automated merging building and testing of code which is committed to a repository.

On a local scale we can use the tox command-line tool to implement a CI like way of working. We can define different envirnoments for which the complete test suite is run whenever we want.

tox can also be integrated in an online CI tool like GitHub Actions. 